<a href="https://colab.research.google.com/github/qianjing2020/DS-Unit-1-Sprint-2-Statistics/blob/master/JingQian_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
#Load and clean the data (or determine the best method to drop observations when running tests)
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp

In [0]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
col_names = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
df = pd.read_csv(url, header=None, names=col_names)             

Number of Instances: 435 (267 democrats, 168 republicans)
Number of Attributes: 16 + class name = 17 (all Boolean valued)
Attribute Information:
   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)
Missing Attribute Values: Denoted by "?"





In [29]:
df.head(4)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y


In [0]:
df = df.replace({'?': np.NaN, 'y': 1, 'n':0})

In [22]:
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [0]:
# Filter the dataframe into a democrats and republicans dataframes
dem = df[df['party']=='democrat']
rep = df[df['party']=='republican']

In [25]:
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


Three main types of t-test:

An Independent Samples t-test compares the means for two groups.

A Paired sample t-test compares means from the same group at different times (say, one year apart).

A One sample t-test tests the mean of a single group against a known mean.

Jing: go read more on https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/t-test/

In [45]:
# 2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
# H0: dem<=rep, Ha: dem>rep
print('The following bills have more democrats support than republicans with p<0.01')
for item in col_names[2:]:
  [t_score, p]=ttest_ind(dem[item], rep[item], nan_policy='omit', equal_var=False)
  if (t_score>0) & (p<0.02):
    print(item)

The following bills have more democrats support than republican with p<0.01
budget
anti-satellite-ban
aid-to-contras
mx-missile
synfuels
duty-free
south-africa


In [46]:
#Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
# H0: rep<=dem, Ha: rep>dem
print('The following bills have more republicans support than democrats with p<0.01')
for item in col_names[2:]:
  [t_score, p]=ttest_ind(rep[item], dem[item], nan_policy='omit', equal_var=False)
  if (t_score>0) & (p<0.02):
    print(item)

The following bills have more republican support than democrat with p<0.01
physician-fee-freeze
el-salvador-aid
religious-groups
education
right-to-sue
crime


In [48]:
#Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
# H0: dem==rep, Ha: dem!==rep
print('The following bills bear no significant differences between democrats and republicans with p>0.1')
for item in col_names[2:]:
  [t_score, p]=ttest_ind(dem[item], rep[item], nan_policy='omit', equal_var=False)
  if p>0.1:
    print(item)

The following bills have more democrats support than republican with p>0.1
water-project


In [0]:
# Stretch goals:
# Refactor your code into functions so it's easy to rerun with arbitrary variables
# Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [61]:
# ttest_multi_columns do a t test on two dataframe with same header

def ttest_multi_columns(df1, df2, alpha):
  v1_name = df1.columns[1]
  v2_name = df2.columns[1]
  
  col_names = df1.columns[1:].values
  for item in col_names:
    [t_score, p]=ttest_ind(df1[item], df2[item], nan_policy='omit', equal_var=False)
    if (t_score>0) & (p<alpha):
      print('There is a significant difference between df1 and df2, p<', alpha, 'on bill', item)
    else: print('No significant difference oberserved between df1 and df2 on bill',item)


ttest_multi_columns(dem, rep, 0.01)

There is a significant difference between df1 and df2, p< 0.01 on bill handicapped-infants
No significant difference oberserved between df1 and df2 on bill water-project
There is a significant difference between df1 and df2, p< 0.01 on bill budget
No significant difference oberserved between df1 and df2 on bill physician-fee-freeze
No significant difference oberserved between df1 and df2 on bill el-salvador-aid
No significant difference oberserved between df1 and df2 on bill religious-groups
There is a significant difference between df1 and df2, p< 0.01 on bill anti-satellite-ban
There is a significant difference between df1 and df2, p< 0.01 on bill aid-to-contras
There is a significant difference between df1 and df2, p< 0.01 on bill mx-missile
No significant difference oberserved between df1 and df2 on bill immigration
There is a significant difference between df1 and df2, p< 0.01 on bill synfuels
No significant difference oberserved between df1 and df2 on bill education
No significan